# Table of contents:

* [Introduction to block ciphers](#intro-block)
* [Padding a message](#message-padding)
* [The Advanced Encryption Standard (AES)](#AES)
* [Modes of operation of block ciphers](#modes)
* [Size of the output ciphertex on AES](#size)
* [Bonus: Fernet cipher](#fernet)
    
Author: [Sebastià Agramunt Puig](https://github.com/sebastiaagramunt) for [OpenMined](https://www.openmined.org/) Privacy ML Series course.



In [10]:
""" # Please remove the triple quotes in this line if the pip installation of the libraries is needed
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
%pip install pycryptodome
%pip install cryptography
%pip install requests --disable-pip-version-check """;

## Block Ciphers <a class="anchor" id="intro-block"></a>

Block ciphers as opposed to stream ciphers take a block of the plaintext (a specific amount of bytes) and encrypts it into a block with the same size. In this section we will use the Advanced Encryption Standard (AES) to understand block ciphers. In the next schema it is shown how an original message of arbitrary $N$ bytes is converted into a ciphertext having blocks of $K$ bytes. The ciphertext size is always a multiple of $K$ bytes.

## Padding a message <a class="anchor" id="message-padding"></a>

Most of the times the lenght of the message is not a multiple of the block size so we need to "pad" the message to have the required length. A common padding function is [PKCS7](https://en.wikipedia.org/wiki/Padding_(cryptography)). Basically what PKCS7 does is appendinng a list of bytes with the same value corresponding to the number of bytes needed to complete the block.




In [11]:
import math
from base64 import b64encode
from sys import getsizeof

msg=b'Information security is a great course !'
def pad(text, block_size):
    no_of_blocks = math.ceil(len(text)/float(block_size))
    pad_len = int(no_of_blocks * block_size - len(text))

    if pad_len == 0:
        return text + (chr(block_size) * block_size).encode()
    else:
        return text + (chr(pad_len) * pad_len).encode()
res=pad(msg,16)
print("PlainText before Padding: "+msg.decode())
print("PlainText after Padding (bytes): "+str((res)))
print("PlainText size: ",getsizeof(msg),' bytes')        #in bytes
print("Padded Text size: ",getsizeof(res),' bytes')    #in bytes

PlainText before Padding: Information security is a great course !
PlainText after Padding (bytes): b'Information security is a great course !\x08\x08\x08\x08\x08\x08\x08\x08'
PlainText size:  73  bytes
Padded Text size:  81  bytes


## Encrypting using AES (Advanced Encryption Standard) <a class="anchor" id="AES"></a>

AES is a block cipher that was established as a standard by NIST in 2001 (after a public call to improve/substitute DES encryption algorithm in 1997). AES is a subset of the Rijndael block cipher developed by Vincent Rijmen and Joan Daemen submitted to NIST during the [AES selection process](https://en.wikipedia.org/wiki/Advanced_Encryption_Standard_process).


We are not going to go into the details of te exact implementation but the readers are referred to the book of [Katz and Lindell](http://www.cs.umd.edu/~jkatz/imc.html) Chapter 6 section 2. Also Mike Pound explains AES in this [video](https://www.youtube.com/watch?v=O4xNJsjtN6E&t=524s&ab_channel=Computerphile), check it out!

In [12]:
from binascii import hexlify, unhexlify

class AES_cipher(object):
    def __init__(self, key, iv=None):
        if isinstance(key, int):
            if abs(key) <= 0xffffffffffffffffffffffffffffffff:
                self.Nb, self.Nk, self.Nr, self.key = 4, 4, 10, "%032x" % key
            elif abs(key) <= 0xffffffffffffffffffffffffffffffffffffffffffffffff:
                self.Nb, self.Nk, self.Nr, self.key = 4, 6, 12, "%048x" % key
            elif abs(key) <= 0xffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff:
                self.Nb, self.Nk, self.Nr, self.key = 4, 8, 14, "%064x" % key
            else:
                raise ValueError("Key can not be larger than 256-bits.")

        self.s_box = [ #S-Box
            0x63, 0x7c, 0x77, 0x7b, 0xf2, 0x6b, 0x6f, 0xc5, 0x30, 0x01, 0x67, 0x2b, 0xfe, 0xd7, 0xab, 0x76,
            0xca, 0x82, 0xc9, 0x7d, 0xfa, 0x59, 0x47, 0xf0, 0xad, 0xd4, 0xa2, 0xaf, 0x9c, 0xa4, 0x72, 0xc0,
            0xb7, 0xfd, 0x93, 0x26, 0x36, 0x3f, 0xf7, 0xcc, 0x34, 0xa5, 0xe5, 0xf1, 0x71, 0xd8, 0x31, 0x15,
            0x04, 0xc7, 0x23, 0xc3, 0x18, 0x96, 0x05, 0x9a, 0x07, 0x12, 0x80, 0xe2, 0xeb, 0x27, 0xb2, 0x75,
            0x09, 0x83, 0x2c, 0x1a, 0x1b, 0x6e, 0x5a, 0xa0, 0x52, 0x3b, 0xd6, 0xb3, 0x29, 0xe3, 0x2f, 0x84,
            0x53, 0xd1, 0x00, 0xed, 0x20, 0xfc, 0xb1, 0x5b, 0x6a, 0xcb, 0xbe, 0x39, 0x4a, 0x4c, 0x58, 0xcf,
            0xd0, 0xef, 0xaa, 0xfb, 0x43, 0x4d, 0x33, 0x85, 0x45, 0xf9, 0x02, 0x7f, 0x50, 0x3c, 0x9f, 0xa8,
            0x51, 0xa3, 0x40, 0x8f, 0x92, 0x9d, 0x38, 0xf5, 0xbc, 0xb6, 0xda, 0x21, 0x10, 0xff, 0xf3, 0xd2,
            0xcd, 0x0c, 0x13, 0xec, 0x5f, 0x97, 0x44, 0x17, 0xc4, 0xa7, 0x7e, 0x3d, 0x64, 0x5d, 0x19, 0x73,
            0x60, 0x81, 0x4f, 0xdc, 0x22, 0x2a, 0x90, 0x88, 0x46, 0xee, 0xb8, 0x14, 0xde, 0x5e, 0x0b, 0xdb,
            0xe0, 0x32, 0x3a, 0x0a, 0x49, 0x06, 0x24, 0x5c, 0xc2, 0xd3, 0xac, 0x62, 0x91, 0x95, 0xe4, 0x79,
            0xe7, 0xc8, 0x37, 0x6d, 0x8d, 0xd5, 0x4e, 0xa9, 0x6c, 0x56, 0xf4, 0xea, 0x65, 0x7a, 0xae, 0x08,
            0xba, 0x78, 0x25, 0x2e, 0x1c, 0xa6, 0xb4, 0xc6, 0xe8, 0xdd, 0x74, 0x1f, 0x4b, 0xbd, 0x8b, 0x8a,
            0x70, 0x3e, 0xb5, 0x66, 0x48, 0x03, 0xf6, 0x0e, 0x61, 0x35, 0x57, 0xb9, 0x86, 0xc1, 0x1d, 0x9e,
            0xe1, 0xf8, 0x98, 0x11, 0x69, 0xd9, 0x8e, 0x94, 0x9b, 0x1e, 0x87, 0xe9, 0xce, 0x55, 0x28, 0xdf,
            0x8c, 0xa1, 0x89, 0x0d, 0xbf, 0xe6, 0x42, 0x68, 0x41, 0x99, 0x2d, 0x0f, 0xb0, 0x54, 0xbb, 0x16]

        self.exp_box = [
            0x01, 0x03, 0x05, 0x0f, 0x11, 0x33, 0x55, 0xff, 0x1a, 0x2e, 0x72, 0x96, 0xa1, 0xf8, 0x13, 0x35,
            0x5f, 0xe1, 0x38, 0x48, 0xd8, 0x73, 0x95, 0xa4, 0xf7, 0x02, 0x06, 0x0a, 0x1e, 0x22, 0x66, 0xaa,
            0xe5, 0x34, 0x5c, 0xe4, 0x37, 0x59, 0xeb, 0x26, 0x6a, 0xbe, 0xd9, 0x70, 0x90, 0xab, 0xe6, 0x31,
            0x53, 0xf5, 0x04, 0x0c, 0x14, 0x3c, 0x44, 0xcc, 0x4f, 0xd1, 0x68, 0xb8, 0xd3, 0x6e, 0xb2, 0xcd,
            0x4c, 0xd4, 0x67, 0xa9, 0xe0, 0x3b, 0x4d, 0xd7, 0x62, 0xa6, 0xf1, 0x08, 0x18, 0x28, 0x78, 0x88,
            0x83, 0x9e, 0xb9, 0xd0, 0x6b, 0xbd, 0xdc, 0x7f, 0x81, 0x98, 0xb3, 0xce, 0x49, 0xdb, 0x76, 0x9a,
            0xb5, 0xc4, 0x57, 0xf9, 0x10, 0x30, 0x50, 0xf0, 0x0b, 0x1d, 0x27, 0x69, 0xbb, 0xd6, 0x61, 0xa3,
            0xfe, 0x19, 0x2b, 0x7d, 0x87, 0x92, 0xad, 0xec, 0x2f, 0x71, 0x93, 0xae, 0xe9, 0x20, 0x60, 0xa0,
            0xfb, 0x16, 0x3a, 0x4e, 0xd2, 0x6d, 0xb7, 0xc2, 0x5d, 0xe7, 0x32, 0x56, 0xfa, 0x15, 0x3f, 0x41,
            0xc3, 0x5e, 0xe2, 0x3d, 0x47, 0xc9, 0x40, 0xc0, 0x5b, 0xed, 0x2c, 0x74, 0x9c, 0xbf, 0xda, 0x75,
            0x9f, 0xba, 0xd5, 0x64, 0xac, 0xef, 0x2a, 0x7e, 0x82, 0x9d, 0xbc, 0xdf, 0x7a, 0x8e, 0x89, 0x80,
            0x9b, 0xb6, 0xc1, 0x58, 0xe8, 0x23, 0x65, 0xaf, 0xea, 0x25, 0x6f, 0xb1, 0xc8, 0x43, 0xc5, 0x54,
            0xfc, 0x1f, 0x21, 0x63, 0xa5, 0xf4, 0x07, 0x09, 0x1b, 0x2d, 0x77, 0x99, 0xb0, 0xcb, 0x46, 0xca,
            0x45, 0xcf, 0x4a, 0xde, 0x79, 0x8b, 0x86, 0x91, 0xa8, 0xe3, 0x3e, 0x42, 0xc6, 0x51, 0xf3, 0x0e,
            0x12, 0x36, 0x5a, 0xee, 0x29, 0x7b, 0x8d, 0x8c, 0x8f, 0x8a, 0x85, 0x94, 0xa7, 0xf2, 0x0d, 0x17,
            0x39, 0x4b, 0xdd, 0x7c, 0x84, 0x97, 0xa2, 0xfd, 0x1c, 0x24, 0x6c, 0xb4, 0xc7, 0x52, 0xf6, 0x01]

        self.ln_box = [
            0x00, 0x00, 0x19, 0x01, 0x32, 0x02, 0x1a, 0xc6, 0x4b, 0xc7, 0x1b, 0x68, 0x33, 0xee, 0xdf, 0x03,
            0x64, 0x04, 0xe0, 0x0e, 0x34, 0x8d, 0x81, 0xef, 0x4c, 0x71, 0x08, 0xc8, 0xf8, 0x69, 0x1c, 0xc1,
            0x7d, 0xc2, 0x1d, 0xb5, 0xf9, 0xb9, 0x27, 0x6a, 0x4d, 0xe4, 0xa6, 0x72, 0x9a, 0xc9, 0x09, 0x78,
            0x65, 0x2f, 0x8a, 0x05, 0x21, 0x0f, 0xe1, 0x24, 0x12, 0xf0, 0x82, 0x45, 0x35, 0x93, 0xda, 0x8e,
            0x96, 0x8f, 0xdb, 0xbd, 0x36, 0xd0, 0xce, 0x94, 0x13, 0x5c, 0xd2, 0xf1, 0x40, 0x46, 0x83, 0x38,
            0x66, 0xdd, 0xfd, 0x30, 0xbf, 0x06, 0x8b, 0x62, 0xb3, 0x25, 0xe2, 0x98, 0x22, 0x88, 0x91, 0x10,
            0x7e, 0x6e, 0x48, 0xc3, 0xa3, 0xb6, 0x1e, 0x42, 0x3a, 0x6b, 0x28, 0x54, 0xfa, 0x85, 0x3d, 0xba,
            0x2b, 0x79, 0x0a, 0x15, 0x9b, 0x9f, 0x5e, 0xca, 0x4e, 0xd4, 0xac, 0xe5, 0xf3, 0x73, 0xa7, 0x57,
            0xaf, 0x58, 0xa8, 0x50, 0xf4, 0xea, 0xd6, 0x74, 0x4f, 0xae, 0xe9, 0xd5, 0xe7, 0xe6, 0xad, 0xe8,
            0x2c, 0xd7, 0x75, 0x7a, 0xeb, 0x16, 0x0b, 0xf5, 0x59, 0xcb, 0x5f, 0xb0, 0x9c, 0xa9, 0x51, 0xa0,
            0x7f, 0x0c, 0xf6, 0x6f, 0x17, 0xc4, 0x49, 0xec, 0xd8, 0x43, 0x1f, 0x2d, 0xa4, 0x76, 0x7b, 0xb7,
            0xcc, 0xbb, 0x3e, 0x5a, 0xfb, 0x60, 0xb1, 0x86, 0x3b, 0x52, 0xa1, 0x6c, 0xaa, 0x55, 0x29, 0x9d,
            0x97, 0xb2, 0x87, 0x90, 0x61, 0xbe, 0xdc, 0xfc, 0xbc, 0x95, 0xcf, 0xcd, 0x37, 0x3f, 0x5b, 0xd1,
            0x53, 0x39, 0x84, 0x3c, 0x41, 0xa2, 0x6d, 0x47, 0x14, 0x2a, 0x9e, 0x5d, 0x56, 0xf2, 0xd3, 0xab,
            0x44, 0x11, 0x92, 0xd9, 0x23, 0x20, 0x2e, 0x89, 0xb4, 0x7c, 0xb8, 0x26, 0x77, 0x99, 0xe3, 0xa5,
            0x67, 0x4a, 0xed, 0xde, 0xc5, 0x31, 0xfe, 0x18, 0x0d, 0x63, 0x8c, 0x80, 0xc0, 0xf7, 0x70, 0x07]

        self.rcon = [0x00, 0x01, 0x02, 0x04, 0x08, 0x10, 0x20, 0x40, 0x80, 0x1B, 0x36]

    @staticmethod
    def pad(data, block=16):
        pad = block - (len(data) % block)
        return data + bytearray(pad for _ in range(pad))

    @staticmethod
    def unpad(data):
        return data[:-data[-1]]

    @staticmethod
    def rotate_word(word):
        """
        Takes a word [a0, a1, a2, a3] as input and perform a
        cyclic permutation that returns the word [a1, a2, a3, a0].
        """
        return ((word << 4) | (word >> (16 - 4))) & 0xffff

    @staticmethod
    def state_matrix(state):
        """
        Formats a State Matrix to a properly formatted list.

        @param state: State Matrix
        @return: Formatted State Matrix
        """
        new_state = []
        for x in range(4):
            new_state += [state[0 + x], state[4 + x], state[8 + x], state[12 + x]]
        return new_state

    @staticmethod
    def inv_state_matrix(state):
        """-
        Preform the inverse of the State matrix method.

        @param state: State Matrix
        @return: Reverted State Matrix
        """
        columns = [state[x:x + 4] for x in range(0, 16, 4)]
        new_state = []
        for x in range(4):
            new_state += [columns[0][x], columns[1][x], columns[2][x], columns[3][x]]
        return new_state

    def galois(self, a, b):
        """
        Galois multiplication of 8 bit characters a and b.

        @param a: State Matrix col or row
        @param b: Fixed number
        @return: Galois field GF(2^8)
        """
        if a != 0 and b != 0:
            return self.exp_box[(self.ln_box[a] + self.ln_box[b]) % 0xff]
        return 0

    def sub_word(self, byte):
        """
        Key Expansion routine that takes a four-byte
        input word and applies an S-box substitution.

        @param byte: Output from the circular byte left shift
        @return: Substituted bytes through s_box
        """
        return ((self.s_box[(byte >> 24 & 0xff)] << 24) + (self.s_box[(byte >> 16 & 0xff)] << 16) +
                (self.s_box[(byte >> 8 & 0xff)] << 8) + self.s_box[byte & 0xff])  

    def sub_bytes(self, state):#1
        """
        Transforms the State Matrix using a nonlinear byte S-box
        that operates on each of the State bytes independently.

        @param state: State matrix input
        @return: Byte substitution from the state matrix
        """
        return [self.s_box[state[x]] for x in range(16)]

    def shift_rows(self, state):#2
        """
        Changes the State by cyclically shifting the last
        three rows of the State by different offsets.

        @param state: State Matrix
        @return: Shifted state by offsets [0, 1, 2, 3]
        """
        offset = 0
        for x in range(0, 16, 4):
            state[x:x + 4] = state[x:x + 4][offset:] + state[x:x + 4][:offset]
            offset += 1
        return state

    def mix_columns(self, state):#3
        """
        Operates on the State column-by-column, treating each column as
        a four-term polynomial. The columns are considered as polynomials
        over GF(2^8) and multiplied modulo x^4 + 1 with a fixed polynomial a(x).

        @param state: State Matrix input
        @return: Byte substitution from the state matrix
        """
        columns = [state[x:x + 4] for x in range(0, 16, 4)]
        output = []
        for x in range(4):
            output.append(self.galois(columns[0][x], 2) ^ self.galois(columns[3][x], 1) ^ self.galois(columns[2][x], 1) ^ self.galois(columns[1][x], 3))
            output.append(self.galois(columns[1][x], 2) ^ self.galois(columns[0][x], 1) ^ self.galois(columns[3][x], 1) ^ self.galois(columns[2][x], 3))
            output.append(self.galois(columns[2][x], 2) ^ self.galois(columns[1][x], 1) ^ self.galois(columns[0][x], 1) ^ self.galois(columns[3][x], 3))
            output.append(self.galois(columns[3][x], 2) ^ self.galois(columns[2][x], 1) ^ self.galois(columns[1][x], 1) ^ self.galois(columns[0][x], 3))
        return self.state_matrix(output)

    @staticmethod
    def add_rKey(state, key):#4
        """
        Round Key is added to the State using an XOR operation.

        @param state: State Matrix
        @param key: Round Key
        @return: Hex values of XOR operation
        """
        return [state[x] ^ key[x] for x in range(16)]

    def cipher(self, expanded_key, data):
        """
        @param expanded_key: The expanded key schedule
        @param data: Data to encrypt
        @return: Encrypted data
        """
        state = self.add_rKey(self.state_matrix(data), expanded_key[0])

        for r in range(self.Nr - 1):
            state = self.sub_bytes(state)
            state = self.shift_rows(state)
            state = self.mix_columns(state)
            state = self.add_rKey(state, expanded_key[r + 1])

        state = self.sub_bytes(state)
        state = self.shift_rows(state)
        state = self.add_rKey(state, expanded_key[self.Nr])
        return self.inv_state_matrix(state)

    def expand_key(self, key):#5
        """
        Takes the Cipher Key and performs a Key Expansion routine to
        generate a key schedule thus generating a total of Nb (Nr + 1) words.

        @param key: Cipher Key
        @return: Expanded Cipher Keys
        """
        w = [int(key[y:y + 8], 16) for y in range(0, len(key), 8)]

        i = self.Nk
        while i < self.Nb * (self.Nr + 1):
            temp = w[i - 1]
            if i % self.Nk == 0:
                temp = self.sub_word(self.rotate_word(temp)) ^ (self.rcon[i // self.Nk] << 24)
            elif self.Nk > 6 and i % self.Nk == 4:
                temp = self.sub_word(temp)
            w.append(w[i - self.Nk] ^ temp)
            i += 1

        new_state = []
        for x in range(0, len(w), 4):
            state = []
            for y in range(4):
                state += [w[x + y] >> 24 & 0xff, w[x + y] >> 16 & 0xff, w[x + y] >> 8 & 0xff, w[x + y] & 0xff]
            new_state.append(self.state_matrix(state))
        return new_state

    def encrypt(self, data):
        """
        Encryption method.

        @param data: Data to be encrypted
        @return: Encrypted data
        """
        expanded_key = self.expand_key(self.key)
        data = self.pad(bytes(data, 'utf-8'))
        blocks = [self.cipher(expanded_key, data[x:x + 16]) for x in range(0, len(data), 16)]
        return hexlify(bytes(y for x in blocks for y in x)).decode()

#Test-Run
key = 5555
msg='Balabizo'
AES_cipher = AES_cipher(key)
ciphertext = AES_cipher.encrypt(msg)
print("PlainText:  ",msg)
print("CipherText: ",ciphertext)

PlainText:   Balabizo
CipherText:  b92b4e0a9eac2396ebc839fa2a30eaba


The AES cipher works using a 128/192/156 bit key and 128 bit data. It is iterative rather than a Feistel cipher. It processes data as block of 4 columns of 4 bytes and operates on entire data block every round.

In its  9/11/13 rounds, the process is as follows:

1. Byte Subsitution (1 S-Box used on every byte)
2. Shift rows 
3. Mix Columns
4. Add Round Key
5. View as alternating XOR key and scramble data bytes

In the above code, we created methods to carry out each of these processes.

In the Byte Subsitution step each byte of state is replaced by byte indexed by row (left 4-bits) & column (right 4-bits).

In the Shift rows step, a circular byte shift happens:

-   1st row is unchanged
-   2nd row does 1 byte circular shift to left
-   3rd row does 2 byte circular shift to left
-   4th row does 3 byte circular shift to left

In the mix columns step each column is processed separately and each byte is replaced by a value dependent on all 4 bytes in the column
It is effectively a matrix multiplication.
The coefficients of the matrix ensure a good mixing among the bytes of each column by maximizing the distances between the code words. The mix column transformation combined with the shift row transformation ensures that after a few rounds all output bits depend on all input bits.

In the add round key step a XOR state with 128-bits of the round key occurs and it is processed by column.

In [13]:
#Example of AES using EAX mode
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from base64 import b64encode

key=b'1234567891234567'
nonce = get_random_bytes(15)    # instead of getting a random nonce
message = "Information security is a great course !".encode() # convert the string to bytes using encode function

# Encryption
cipher = AES.new(key, AES.MODE_EAX, nonce=nonce)
ciphertext, mac = cipher.encrypt_and_digest(message)
# Decryption
cipher = AES.new(key, AES.MODE_EAX, nonce=nonce)
plaintext = cipher.decrypt_and_verify(ciphertext, mac)

print("PlainText before Encryption: "+message.decode())
print("Ciphertext: "+b64encode(ciphertext).decode())
print("Back to PlainText: "+plaintext.decode())

PlainText before Encryption: Information security is a great course !
Ciphertext: m/isgwJK/SKEP1Clenyd1C/Drjo7Bvf82eXFsbDZnxkRv8YZw5KCEQ==
Back to PlainText: Information security is a great course !


## Modes of operation of block ciphers <a class="anchor" id="mode"></a>

A block cipher by itself is only suitable for the secure cryptographic transformation (encryption or decryption) of one fixed-length group of bits called a block. A mode of operation describes how to repeatedly apply a cipher's single-block operation to securely transform amounts of data larger than a block ([Wikipedia](https://en.wikipedia.org/wiki/Block_cipher_mode_of_operation)).

The first mode is "not doing anything", this is the Electronic Codebook mode. See the figure below (from Wikipedia).
We are lucky and in ```cryptography``` package ECB implemented in ```cryptography.hazmat.primitives.ciphers.ECB``` function (we've seen in the previous example!).
Now we can encrypt the same message twice and see what we get in the ciphertext:

# AES USING ECB (ELECTRONIC CODE BOOK)

In the ECB process conducted below, the message is first broken down into independent blocks which are encrypted. Each block is a value which is subsituted, like a codebook, hence its name.

In [14]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad, unpad
from base64 import b64encode
for x in range(2):
    # key = get_random_bytes(16)      # A 16 byte key for AES-128
    key=b'1234567891234567'
    message = "Information security is a great course, I love it !".encode() # convert the string to bytes using encode function

    # Encryption
    cipher = AES.new(key, AES.MODE_ECB)
    ciphertext= cipher.encrypt(pad(message, AES.block_size)) # Block size= 16 bytes
    # Decryption
    decrypt_cipher = AES.new(key, AES.MODE_ECB)
    plaintext =  decrypt_cipher.decrypt(ciphertext)
    print("Ciphertext "+str(x)+" : "+b64encode(ciphertext).decode())
    print("Back to PlainText "+str(x)+" : "+plaintext.decode()+'\n')

Ciphertext 0 : TLHenMawihtNigFCM8Ub0mDIBGYS4iN33JaUOLJ6/xeW9hrNVPnMM6ooHIZdZH9P3JUNHQEy2+UwjdMz8WQNmA==
Back to PlainText 0 : Information security is a great course, I love it !

Ciphertext 1 : TLHenMawihtNigFCM8Ub0mDIBGYS4iN33JaUOLJ6/xeW9hrNVPnMM6ooHIZdZH9P3JUNHQEy2+UwjdMz8WQNmA==
Back to PlainText 1 : Information security is a great course, I love it !



As we can see running the encryption algorithm n-times will result in the same ciphertext.
This is not a desirable outcome. If I want to send the same message twice, I really don't want to send the same ciphertext. What if in all comunications I start by "Dear..." and the attacker knows it?. 

A better mode is the Cipher block chaining (CBC):

# AES USING CBC (CIPHER BLOCK CHAINING)

In the CBC Mode implemented below, the message is also broken into blocks. However, unlike the ECB, they are linked together in encryption operation where each previous cipher block is chained with current plaintext block. It also differs in that it uses an Initial Vector (IV) to start the process.

In [15]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad, unpad
from base64 import b64encode
for x in range(2):
    #Key size can be 16, 24 or 32 bytes 
    # key = get_random_bytes(16)      # A 16 byte key for AES-128
    key=b'1234567891234567' # 16 bytes
    m = "Information Security is the best course, I love it !" 
    message=m.encode()      # convert the string to bytes using encode function
    # Encryption
    cipher = AES.new(key, AES.MODE_CBC)
    ciphertext= cipher.encrypt(pad(message, AES.block_size)) # Block size= 16 bytes
    iv = cipher.iv
    # Decryption
    decrypt_cipher = AES.new(key, AES.MODE_CBC,iv)
    plaintext =  decrypt_cipher.decrypt(ciphertext)
    print("Ciphertext ",str(x)+" : "+b64encode(ciphertext).decode())
    print("Back to PlainText "+str(x)+" : "+plaintext.decode()+'\n')

Ciphertext  0 : CRvNNlOj4dapBh+yavMVDOLRPW6cvdWY2XNids7AjNMUZL3n2Uez5vB/K21o8rGdoXjzBkfGCkKTtgdLra8gQw==
Back to PlainText 0 : Information Security is the best course, I love it !

Ciphertext  1 : 8s3CMihG/7sDf7LFjSDWZq998uu7JkMGm71pO+CeRYW8u/QOPxLc0wUGs4bg/IKKTfflnLdCa4b6F8P68io4cQ==
Back to PlainText 1 : Information Security is the best course, I love it !



In this case we take a random initialization vector and perform XOR operation with the block of plaintext, then we feed this into the encryptor, after that we obtain the ciphertext. This ciphertext is used as the initialization vector to encrypt the next block.

## Size of ciphertext <a class="anchor" id="size"></a>

In [16]:
import sys
print("CipherText-Data"+'\n'+'-'*100)
print("CipherText: "+str(b64encode(ciphertext).decode()))
print("Length of CipherText: "+str(len(ciphertext)))
print("Size of CipherText: "+str(sys.getsizeof(ciphertext)),' bytes'+'\n')
print("PlainText-Data"+'\n'+'-'*100)
print("PlainText: "+m)
print("Length of PlainText: "+str(len(m)))
print("Size of PlainText: "+str(sys.getsizeof(m)),' bytes'+'\n')

CipherText-Data
----------------------------------------------------------------------------------------------------
CipherText: 8s3CMihG/7sDf7LFjSDWZq998uu7JkMGm71pO+CeRYW8u/QOPxLc0wUGs4bg/IKKTfflnLdCa4b6F8P68io4cQ==
Length of CipherText: 64
Size of CipherText: 97  bytes

PlainText-Data
----------------------------------------------------------------------------------------------------
PlainText: Information Security is the best course, I love it !
Length of PlainText: 52
Size of PlainText: 101  bytes



The ciphertext has a size of 97 bytes, while the plaintext has a size of 101 bytes

# Conclusion

The biggest difference observed between the 2 modes (ECB and CBC) is that in the example of ECB we had a set key (just for the sake of this project) that produces the same ciphertext everytime. This is not good, because if someone knows or learns a certain pattern in our plaintext (message), it can be easily decrypted. In the CBC example however, while also setting a fixed key (again just for this example) it produced 2 different ciphertexts. This is better because it adds another layer of security

## Bonus: Fernet <a class="anchor" id="fernet"></a>

Another block cipher implemented in cryptography package is [Fernet](https://asecuritysite.com/encryption/fernet). 

In [17]:
from cryptography.fernet import Fernet
key = Fernet.generate_key()
f = Fernet(key)
token = f.encrypt(b"This bonus is not easy !")
print("Key= "+str(key.decode())+'\n')
print("Token= "+str(token.decode())+'\n')
print("After decryption= "+str(f.decrypt(token).decode()))

Key= raDJgZeJ-W12S0R_NeUM7ULWvc0Io7oIezxYb_HABlQ=

Token= gAAAAABjenU40thxXZI8RaAKq1YL_u5FHS5SG2LhJ-D4EPC2CWyPt_WNeA4Ms3E33mPaOBrsL8RVCa4qPXeOaUX8pXq2vjP6CsbIDLYtUYc6e6zYODY66XI=

After decryption= This bonus is not easy !


Just to double check that our code is running correctly, we will have a fixed key and token and run them on the [Fernet](https://asecuritysite.com/encryption/ferdecode) website and compare the results

In [18]:
from cryptography.fernet import Fernet
# key = Fernet.generate_key()
key=b'e-n2_qyYvwWwXksiSBXUYVvs-eevERo78RuPxwbr-6Y='
f = Fernet(key)
# token = f.encrypt(b"This bonus is not easy !")
token=b'gAAAAABjd9Rn6g-s8YNq0gzFNcxUyMdVrETC5-Apzi_EcqJ0pLAQI_jRx6JJDQap7YO8jRC74wnH1AFELYbQyQsVVCpbQC1tD6BhFsFBA4UBEaJKdEFXG-A='
print("Key= "+str(key.decode())+'\n')
print("Token= "+str(token.decode())+'\n')
print("After decryption = "+str(f.decrypt(token).decode()))

Key= e-n2_qyYvwWwXksiSBXUYVvs-eevERo78RuPxwbr-6Y=

Token= gAAAAABjd9Rn6g-s8YNq0gzFNcxUyMdVrETC5-Apzi_EcqJ0pLAQI_jRx6JJDQap7YO8jRC74wnH1AFELYbQyQsVVCpbQC1tD6BhFsFBA4UBEaJKdEFXG-A=

After decryption = This bonus is not easy !


The results are:

<img src="Fernet.png" style="width:900px"/>

As we can see the key and token resulted in the right plaintext to be decoded, which means that our code is running correctly 